# Tutorial 2:  Building and training Random Forest Models

**Week 2, Day 5: AI and Climate Change**

**By Climatematch Academy**

__Content creators:__  Deepak Mewada, Grace Lindsay

__Content reviewers:__ Jenna Pearson

__Content editors:__ Name Surname, Name Surname

__Production editors:__ Konstantine Tsafatinos

___

# Tutorial Objectives

*Estimated timing of tutorial: 35 minutes*

In this tutorial, we will...   
* Learn about decision trees and hyperparamters
* Learn about random forest models
* Understand how regression models are evaluated (R^2)
* Introduce scikit-learn



In [ ]:
import sys
print(sys.version)


In [ ]:
# @title Tutorial slides

# @markdown These are the slides for the videos in all tutorials today


## Uncomment the code below to test your function

#from IPython.display import IFrame
#link_id = "<YOUR_LINK_ID_HERE>"

print("If you want to download the slides: 'Link to the slides'")
      # Example: https://osf.io/download/{link_id}/

#IFrame(src=f"https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{link_id}/?direct%26mode=render", width=854, height=480)



---
# **Setup**

<details>
<summary> <font color='LightGray'>Click here for overview of this section  </font></summary>

1. **Gadget (optional)**: install and import feedback gadget - not relevant for cliamtematch
2. **Import cell (required)**: imports all libraries you use in the tutorial
3. **Hidden Figure settings cell (optional)**: sets up the plotting style (copy exactly).
4. **Hidden Plotting functions cell (optional)**: should contain all functions used to create plots throughout the tutorial (so students don't waste time looking at boilerplate matplotlib but can here if they wish to). Please use only matplotlib for plotting for consistency.
5. **Hidden Data retrieval (optional)**: set up the data retrieval needed for the tutorial.
6. **Hidden Helper functions cell (optional)**: This should contain functions that students have previously used or that are very simple. Any helper functions that are being used for the first time and are important should be placed directly above the relevant text or exercise (see Tutorial 1.1 for an example).
7. **set random seed (with/out pytorch) and set GPU/CPU (optional)**: several cells that set the seed for repeatability and set the GPU for computations  


In [ ]:
# @title Import necessary libraries:

import matplotlib.pyplot as plt    # For plotting graphs
import pandas as pd                 # For data manipulation
import ipywidgets as widgets  # interactive display
from sklearn.ensemble import RandomForestRegressor        # For Random Forest Regression
from sklearn.tree import DecisionTreeRegressor            # For Decision Tree Regression
from sklearn.tree import plot_tree                        # For plotting decision trees

<details>
<summary> <font color='Red'>Click here if you are running on local machine or you encounter any error while importing   </font></summary>
Please note that if you are running this code on a local machine and encounter an error while importing a library, make sure to install the library via pip. For example, if you receive a `ModuleNotFoundError: No module named 'library name` error , please run `pip install 'library name` to install the required module.

In [ ]:
# @title Set random seed { display-mode: "form" }

# @markdown Executing `set_seed(seed=seed)` you are setting the seed

# E.g., for DL its critical to set the random seed so that students can have a
# baseline to compare their results to expected results.
# Read more here: https://pytorch.org/docs/stable/notes/randomness.html

# Call `set_seed` function in the exercises to ensure reproducibility.
import random
import numpy as np

def set_seed(seed=None):
  if seed is None:
    seed = np.random.choice(2 ** 32)
  random.seed(seed)
  np.random.seed(seed)
  print(f'Random seed {seed} has been set.')


set_seed(seed=2024)  # change 2023 with any number you like
# Set a global seed value for reproducibility
random_state = 42

---
# **Section 1: Preparing the Data for Model Training**
---





In [ ]:
# @title Video 1: Video 1 Name
from ipywidgets import widgets
from IPython.display import display, IFrame, YouTubeVideo

out2 = widgets.Output()
with out2:
  class BiliVideo(IFrame):
    def __init__(self, id, page=1, width=400, height=300, **kwargs):
      self.id=id
      src = 'https://player.bilibili.com/player.html?bvid={0}&page={1}'.format(id, page)
      super(BiliVideo, self).__init__(src, width, height, **kwargs)

  video = BiliVideo(id="", width=854, height=480, fs=1)
  print(f'Video available at https://www.bilibili.com/video/{video.id}')
  display(video)

out1 = widgets.Output()
with out1:
  video = YouTubeVideo(id="", width=854, height=480, fs=1, rel=0)
  print(f'Video available at https://youtube.com/watch?v={video.id}')
  display(video)

out = widgets.Tab([out1, out2])
out.set_title(0, 'Youtube')
out.set_title(1, 'Bilibili')

display(out)

In this video, we learned about:

1. Using regression for prediction tasks, like the one we have.
2. The conceptual understanding of decision trees and their regression capabilities.
3. Random forests as an ensemble of decision trees.
4. Training our model
4. Measuring model performance.
5. Utilizing the scikit-learn toolbox for regression tasks.


## Section 2.1 : Loading the data

Remember from the previous tutorial how we loaded the in `training_data`?
let's again load the data here for this tutorial.

In [ ]:
#Load Dataset
url_Climatebench_train_val = "https://osf.io/y2pq7/download"
training_data = pd.read_csv(url_Climatebench_train_val)

Next, We will first prepare the data so that we can train a model to predict temperature in 2050. Let's also remind ourselves of what the data contains:

## Section 2.2 : Preparing the data

In [ ]:
# Check column names (assuming a pandas DataFrame)
print("Column names:")
print(training_data.columns.tolist())  # List all column names

First, we will drop the `scenario` column from the data as it is just a label, but will not be passed into the model.

In [ ]:
training_data.pop('scenario')

As we can see, scenario is no longer in the dataset:

In [ ]:
print("Column names:")
print(training_data.columns.tolist())  # List all column names



Now next we need to pull out our target variable (that is, the variable we want our model to predict). Here that is `tas_FINAL`, the temperature in 2050.  

In [ ]:
target = training_data.pop('tas_FINAL')

Note: we will need to repeat these preprocessing steps anytime we load this (or other) data

# Section 2 : Fit Decision Tree and Random Forest

Now we can train our models. As mentioned in the video, Decision Trees and Random Forest Models can both do regression. Specifically:

Decision Tree Regression:   
* Decision trees recursively partition the feature space into regions based on feature values to predict the target variable.
* Each leaf node represents a prediction.
* Single trees can be prone to capturing noise in the data (not what we want!).  

Random Forest Regression:   
* An ensemble method that combines multiple decision trees to improve predictive performance.
* Each tree is trained on a random subset of the data.
* Aggregates predictions of individual trees to improve performance.
* Typically more robust/doesn't capture noise.

We will see an example of both here.



First let's train a single decision to try to predict 2050 temperature using 2015 temperature and emissions data. We can control the depth of our decision tree (which is the maximum number of splits performed), which we will set to 20 here.

## Section 2.0: Scikit-learn

In this and coming sub-sections, we will utilize [Scikit-learn](https://scikit-learn.org/stable/), commonly referred to as sklearn, a renowned Python library extensively employed for machine learning endeavors. It provides a comprehensive array of functions and tools tailored for various machine learning tasks. Specifically, we will concentrate on the [`DecisionTreeRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) and [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) modules offered by Scikit-learn.

## Section 2.1: Training the Decision Tree and Analyzing the Results

In [ ]:
#instantiate the model:
dt_regressor = DecisionTreeRegressor(random_state=random_state,max_depth=20)

#fit/train the model with the data:
dt_regressor.fit(training_data, target) #pass in the model inputs and the target it should predict

We've trained our first model! Now let's see how well it performs. As discussed in the video, we will use the coefficient of determination (also known as the R-squared value) as the measure of how well the model is doing.


We can get this value by calling the `score` function and providing the data we want the score calculated on. Here we will evaluate the model on the same data it was trained on.

 <details>
<summary> <font color='lightGreen'>Learn more about the R-Squared value and Coefficient of determination   </font></summary>


 The **R-squared** value indicates the proportion of the variance in the target variable that is predicted from the model.

Specifically, the ***coefficient of determination*** is calculated using the formula:

$$
\color{#3182CE}{R^2} = 1 - \frac{\color{#DC3912}{SS_{\text{residual}}}}{\color{#FF9900}{SS_{\text{total}}}}
$$

where:
- $\color{#FF9900}{SS_{\text{total}}}$ represents the total sum of squares, calculated as the sum of squared differences between the target variable $\color{#2CA02C}{y}$ and its mean $\color{#2CA02C}{\bar{y}}$:

$$
\color{#FF9900}{SS_{\text{total}}} = \sum_{i=1}^{n} (\color{#2CA02C}{y_i} - \color{#2CA02C}{\bar{y}})^2
$$

- $\color{#DC3912}{SS_{\text{residual}}}$ denotes the residual sum of squares, computed as the sum of squared differences between the observed target values $\color{#2CA02C}{y}$ and the predicted values $\color{#FF5733}{\hat{y}}$ provided by the model:

$$
\color{#DC3912}{SS_{\text{residual}}} = \sum_{i=1}^{n} (\color{#2CA02C}{y_i} - \color{#FF5733}{\hat{y}_i})^2
$$

The $\color{#3182CE}{R^2}$ score thus quantifies the proportion of variance in the target variable that is predictable from the independent variables in the model.

This value ranges from 0 to 1, where 1 indicates a perfect fit, meaning the model explains all the variability in the target variable.


In [ ]:
dt_regressor.score(training_data, target)


Now, let's create a scatter plot to compare the true temperature values in 2050 to those predicted by the model:


In [ ]:
# @title Scatter Plot: Predicted vs. True Temperatures for Decision Tree

#get predicted values
predicted = dt_regressor.predict(training_data)

# Create scatter plot
plt.scatter(predicted,target,color='b',label='Predicted vs. True Temperatures')
plt.plot([0,4],[0,4],color='r', label='Ideal Line') # Add a diagonal line for reference
plt.xlabel('Predicted Temperatures')
plt.ylabel('True Temperatures')
plt.legend()
plt.title('Predicted vs. True Temperatures', fontsize=14)

<details>
<summary> <font color='skyblue'>What can we conclude from this score and the scatter plot?<br>
First, pause and think by yourself. Then, compare it with the information provided here:
</font></summary>



As we can see, the model achieves a high score of ~0.9984 on the training data. This indicates that the model can explain approximately 99.84% of the variance in the target variable based on the features in the training dataset. Such a high score suggests that the model fits the training data very well and can effectively capture the underlying patterns or relationships between the features and the target variable. We can see the close alignment between the true value and the value predicted by the model in the plot.

However, it's essential to note that achieving a high score on the training data does not guarantee the model's performance on unseen data (i.e., the test or validation datasets). We will explore this more in the next tutorial.
</details>


---

### Interactive Demo 2.1: Variation in Performance with depth | Visualizing Decision Trees  and Scatter plot

In this interactive demo, we'll visualize decision trees using a widget. This widget enables interactive exploration of decision trees by adjusting two parameters:   
`max_depth` controls the tree's complexity during training, while `dt_vis_depth` determines the depth of the tree to visualize. It dynamically trains a decision tree regressor based on `max_depth`, evaluates its performance with a scatter plot, and visualizes the tree structure up to `dt_vis_depth` using the plot_tree function.    
This allows users to balance model complexity and interpretability, gaining insights into how different depths affect predictive accuracy and tree structure.

In [ ]:
# @title  { run: "auto", display-mode: "form" }
# @markdown Make sure you execute this cell to enable the widget!
#Don't worry about understanding this code! It's to setup an interactive plot.

# Function to train decision tree and display scatter plot
def train_and_plot(max_depth, visualize_depth):
    global dt_regressor, training_data

    # Instantiate and train the decision tree regressor
    dt_regressor = DecisionTreeRegressor(max_depth=max_depth)
    dt_regressor.fit(training_data, target)

    # Calculate and print the score
    score = dt_regressor.score(training_data, target)
    print(f"Model Score: {score}")
    print(f"Please wait for ~{visualize_depth+visualize_depth/2} sec for the figure to render")
    # Generate scatter plot: Predicted vs. True Temperatures
    predicted = dt_regressor.predict(training_data)
    fig, axes = plt.subplots(1, 2, figsize=(15+pow(1.3,visualize_depth), 6+pow(1.2,visualize_depth)), gridspec_kw={'width_ratios': [1, 1+visualize_depth/4]})

    # Scatter plot
    axes[0].scatter(predicted, target, color='blue', alpha=0.7, label='Predicted vs. True Temperatures', edgecolors='black')
    axes[0].plot([min(target), max(target)], [min(target), max(target)], color='red', linestyle='--', label='Ideal Prediction Line')
    axes[0].set_xlabel('Predicted Temperature (°C)', fontsize=12)
    axes[0].set_ylabel('True Temperature (°C)', fontsize=12)
    axes[0].set_title('Comparison of Predicted and True Temperatures', fontsize=14)
    axes[0].legend()
    axes[0].grid(True)

    # Decision tree visualization
    plot_tree(dt_regressor, feature_names=training_data.columns, filled=True, fontsize=8, max_depth=visualize_depth, ax=axes[1])
    axes[1].set_title(f'Decision Tree Visualization (Train_max_depth = {max_depth}, dt_visualize_depth = {visualize_depth})')

    plt.tight_layout()
    plt.show()

# Interactive widget to control max_depth
@widgets.interact(max_depth=(1, 31, 1), dt_vis_depth=(1, 10, 1))
def visualize_tree_with_max_depth(max_depth=20, dt_vis_depth=3):
    train_and_plot(max_depth, dt_vis_depth)


### Interactive Demo 2.1 Discussion

1. How does changing the max_depth parameter affect the decision tree's predictive accuracy and complexity?  

2. What insights can be gained by visualizing the decision tree at different depths (dt_vis_depth)?

3. What patterns or trends do you observe in the residuals (differences between predicted and true temperatures) on the scatter plot? How can these insights guide adjustments to improve the model's predictive accuracy?

In [ ]:
#to_remove_explanation

"""
Discussion:
1. Adjusting the `max_depth` parameter influences the complexity of the decision tree model.
Increasing `max_depth` may lead to a more complex model that can capture intricate patterns in the training data,
potentially resulting in higher predictive accuracy. (However, as we will discuss in the next tutorial, this can also increase the risk of overfitting,
where the model learns noise in the training data instead of true patterns, leading to poor generalization to unseen data.)

2. Visualizing the decision tree at different depths (`dt_vis_depth`) provides insights into the hierarchy of features
and decision-making process within the model. Lower depths reveal high-level splits that capture broader patterns in the data,
while higher depths expose finer details and nuances. By adjusting `dt_vis_depth`,
users can focus on specific branches of the tree, uncovering key decision points and feature interactions.
This exploration helps in understanding how the model makes predictions and identifying influential features in the dataset.

3. By examining the scatter plot, we can identify any consistent patterns or trends in the residuals, indicating potential systematic errors
or biases in the model's predictions. These observations can inform adjustments to the model, such as incorporating additional features
or refining existing ones, to enhance its accuracy. Identifying outliers
or clusters of residuals also highlights areas where the model may struggle to generalize,
suggesting targeted improvements for better performance.
"""

## Section 2.2: Training the Random forest and Analyzing the Results

Now we will train an ensemble of decisions trees, known as a random forest. For this we can use the built-in `RandomForestRegressor` from the [sklearn.ensemble.RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html), which we have already imported.

In [ ]:
rf_regressor = RandomForestRegressor(random_state=random_state)

The line of code creates a random forest regressor object named `rf_regressor`. This regressor is configured to use a specified `random_state` parameter, ensuring that the random number generation process within the algorithm is consistent across different runs. This helps maintain reproducibility in our experiments and ensures consistent results.

Now you will train the model on our data and calculate its score on the same data. Create a plot like the one above in order to visually inspect its performance

### Coding Exercise 2.2: Model Training and Performance Visualization of Ranodm Forest

In this exercise, you will train a random forest regressor model on your data and evaluate its performance by calculating its score on the same data. Additionally, you will create a scatter plot to visually inspect its performance.

In [ ]:
def fit_and_visualize_rf(training_data, target):
    """Fit a random forest regressor to the training data and visualize the results.

    Args:
        training_data (array-like): Input data for training the model.
        target (array-like): Target variable for training the model.

    Returns:
        None
    """

    #################################################
    ## TODO for students: Fit the random forest regressor and visualize the results ##
    # Fill remove the following line of code one you have completed the exercise:
    raise NotImplementedError("Student exercise: Fit the random forest regressor and visualize the results")
    #################################################

    # Fit the random forest regressor to the training data
    #rf_regressor.fit(...., ....)

    # Print the R-squared score of the model
    #print(......)

    # Predict the target variable using the trained model
    predicted = rf_regressor.predict(training_data)

    # Create scatter plot
    plt.scatter(predicted,target,color='b',label='Predicted vs. True Temperatures')
    plt.plot([0,4],[0,4],color='r', label='Ideal Line') # Add a diagonal line for reference
    plt.xlabel('Predicted Temperatures')
    plt.ylabel('True Temperatures')
    plt.legend()
    plt.title('Predicted vs. True Temperatures', fontsize=14)

## Uncomment the code below to test your function
# fit_and_visualize_rf(training_data, target)


In [ ]:
# to_remove solution

def fit_and_visualize_rf(training_data, target):
    """Fit a random forest regressor to the training data and visualize the results.

    Args:
        training_data (array-like): Input data for training the model.
        target (array-like): Target variable for training the model.

    Returns:
        None
    """

    # Fit the random forest regressor to the training data
    rf_regressor.fit(training_data, target)

    # Print the R-squared score of the model
    print(rf_regressor.score(training_data, target))

    # Predict the target variable using the trained model
    predicted = rf_regressor.predict(training_data)

    # Create scatter plot
    plt.scatter(predicted,target,color='b',label='Predicted vs. True Temperatures')
    plt.plot([0,4],[0,4],color='r', label='Ideal Line') # Add a diagonal line for reference
    plt.xlabel('Predicted Temperatures')
    plt.ylabel('True Temperatures')
    plt.legend()
    plt.title('Predicted vs. True Temperatures', fontsize=14)

## Uncomment the code below to test your function

fit_and_visualize_rf(training_data, target)


It seems like our models are performing very well! Let's think a bit more in the next tutorial about what else we should do to evaluate our models...





---


# **Summary**

Estimated timing of tutorial: 35 minutes

In this tutorial, we delved into Random Forest Models and their application in climate prediction. We gained an understanding of regression and how Random Forests combine decision trees to improve predictive accuracy. Through practical exercises, we learned how to evaluate model performance and implement Random Forests using tools like scikit-learn.

---






---


                                Congratulations! You have reached the end of the  tutorial.   



---
